# Setup

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from lib.FFNNClassifier import FFNNClassifier

from lib.MLPLib import MLPLIB
from lib.Utils import model_comparison


In [2]:
def one_hot_encode(y, num_classes=10):
    y = np.asarray(y, dtype=int)
    one_hot = np.zeros((len(y), num_classes), dtype=int)
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

In [3]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Type conversion
X = X.astype('float32')
y = y.astype('int')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode labels
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [4]:
"""
DEFAULT PARAMETERS
"""

lower_bound=5.39294405e-05
upper_bound=1
mean=5.39294405e-05
std=.44
seed=69
hidden_layer_sizes=[128,64,32]
max_iter=15
init_method="normal"
learning_rate_init=0.01
batch_size=50
activation_mlplib="logistic"
activation_ffnn="sigmoid"


# Pengaruh depth (banyak layer) dan width (banyak neuron per layer)

In [5]:
# Configuration for depth variation (keeping width constant)
depth_configs = [
    [10, 10],  # 2 layers
    [10, 10, 10],  # 3 layers
    [10, 10, 10, 10]  # 4 layers
]

# Configuration for width variation (keeping depth constant)
width_configs = [
    [5],  # narrow layer
    [15],  # medium layer
    [30]  # wide layer
]

In [6]:
# Experiment for depth variations (keeping width fixed)
print("Depth Variations Experiment:")
for depth_config in depth_configs:
    print(f"\nTesting depth configuration: {depth_config}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=depth_config,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=depth_config,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(depth_config) + ['softmax']
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Depth Variations Experiment:

Testing depth configuration: [10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.6517142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.6517142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing depth configuration: [10, 10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.4722142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.4722142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing depth configuration: [10, 10, 10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.24035714285714285

[From Scratch FFNNClassifier]
Accuracy:
 0.24035714285714285

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



In [8]:
print("Width Variations Experiment:")
for width in width_configs:
    print(f"\nTesting width configuration: {width}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=width,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=width,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(width) + ['softmax']
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Width Variations Experiment:

Testing width configuration: [5]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.7432142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.7432142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing width configuration: [15]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8022142857142858

[From Scratch FFNNClassifier]
Accuracy:
 0.8022142857142858

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing width configuration: [30]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8432857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8432857142857143

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



# Pengaruh fungsi aktivasi hidden layer

In [5]:
activation_configs = [
    ('linear','identity'),
    ('relu','relu'),
    ('sigmoid','logistic'),
    ('tanh','tanh')
]

In [6]:
print("Width Variations Experiment:")
for act_custom, act_sklearn in activation_configs:
    print(f"\nTesting activation configuration: {act_custom}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=act_sklearn,
        verbose=False
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[act_custom] * len(hidden_layer_sizes) + ['softmax']
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Width Variations Experiment:

Testing activation configuration: linear
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8857857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8857857142857143

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: relu
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.7580714285714286

[From Scratch FFNNClassifier]
Accuracy:
 0.7580714285714286

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: sigmoid
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: tanh
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8095

[From Scratch FFNNClassifier]
Accuracy:
 0.8095

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



# Pengaruh learning rate

In [15]:
learning_rates = [0.1, 0.06, 0.006, 0.0009]

In [16]:
print("Width Variations Experiment:")
for rate in learning_rates:
    print(f"\nTesting activation configuration: {rate}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax']
    )

    model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Width Variations Experiment:

Testing activation configuration: 0.1
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: 0.06
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: 0.006
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: 0.0009
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



# Perbandingan Weight

In [17]:
weight_configs =  ['normal', 'zero', 'uniform']

In [19]:
print("Width Variations Experiment:")
for weight_config in weight_configs:
    print(f"\nTesting weight configuration: {weight_config}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=weight_config,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=weight_config,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax']
    )


    model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Width Variations Experiment:

Testing weight configuration: normal
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing weight configuration: zero
[SKLearn MLPClassifier]
Accuracy:
 0.11428571428571428

[From Scratch FFNNClassifier]
Accuracy:
 0.11428571428571428

[Comparison Result]
[[2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 ...
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]] != [[3.6016149e-06 3.60